<a href="https://colab.research.google.com/github/SwagataJ/football-ad-brand-identification/blob/main/Football_Adboard_Complete_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!python "/content/drive/MyDrive/Football-Adboard/setup.py"

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-28aivjjb
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-28aivjjb
  Resolved https://github.com/facebookresearch/detectron2.git to commit 57bdb21249d5418c130d54e2ebdc94dda7a4c01a
  Preparing metadata (setup.py) ... done
fatal: destination path 'U-2-Net' already exists and is not an empty directory.
fatal: destination path 'glass-text-spotting' already exists and is not an empty directory.
--2023-07-28 05:25:06--  https://glass-text-spotting.s3.eu-west-1.amazonaws.com/models/glass_250k_icdar15_fintune.pth
Resolving glass-text-spotting.s3.eu-west-1.amazonaws.com (glass-text-spotting.s3.eu-west-1.amazonaws.com)... 52.92.34.154, 52.218.46.18, 52.218.25.248, ...
Connecting to glass-text-spotting.s3.eu-west-1.amazonaws.com (glass-text-spotting.s3.eu-west-1.amazonaws.com)|52.92.34.154|:443... connected.
HTTP request sent, awaiting resp

In [ ]:
%cd U-2-Net

/content/U-2-Net


In [ ]:
import torch

assert torch.cuda.is_available(), "Please choose a GPU enabled machine"

In [ ]:
from model import U2NET

model = torch.load("/content/drive/MyDrive/Football-Adboard/u2netsu2net_bce_itr_2938_train_0.091297_tar_0.008402.pth")
net = U2NET(3, 1)
if torch.cuda.is_available():
    net.cuda()
net.load_state_dict(model)

<All keys matched successfully>

In [ ]:
%cd glass-text-spotting/

/content/U-2-Net/glass-text-spotting


In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import math
import json

import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import io
from torchvision.transforms import functional as fn
from torchvision.transforms import (
    functional as fn, Compose,ToPILImage,ToTensor,Resize )

from glass.evaluation.text_evaluator import get_instances_text
from glass.inference.glass_runner import GlassRunner


from wcmatch import glob, pathlib
from statistics import mode
from rapidfuzz.fuzz import token_set_ratio, partial_ratio
from rapidfuzz.process import extractOne, extract

import warnings
warnings.filterwarnings("ignore")

In [ ]:
video_path = "/content/drive/MyDrive/Football-Adboard/Test_video.mp4"
BATCH_SIZE = 1
counter = 1

In [ ]:
databaseFile = open('/content/drive/MyDrive/Football-Adboard/brand_database.json')
database = json.load(databaseFile)

adBoardItems = database.keys()

In [ ]:
def variance_of_laplacian(image):
    '''
    function to calculate laplacian variance for blur detection

    :param image:
    :return: variance value
    '''
    return cv2.Laplacian(image, cv2.CV_64FC1).var()

In [ ]:
def extract_frame(video, fps, start, durationInSeconds, size):

    print(durationInSeconds)
    second = start
    success = 1

    best_frame = video.read()

    image_batch = []
    while second < durationInSeconds and success:
        best_var = 0

        for i in range(math.floor(fps)):
            success, image = video.read()
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            curr_var = variance_of_laplacian(gray)

            if curr_var > best_var:
                best_var = curr_var
                best_frame = image

        image_batch.append(cv2.cvtColor(best_frame,cv2.COLOR_BGR2RGB))

        # if len(image_batch) == BATCH_SIZE:
        #     image_batch = np.asarray(image_batch)
        #     send_to_seg_model(image_batch,size)

        #     image_batch = []

        second += 1
    send_to_seg_model(image_batch, size)

In [ ]:
def send_to_seg_model(batch,size):
    batch_segment = np.zeros_like(batch)
    for c,i in enumerate(batch):
      image_og = torch.from_numpy(i).permute(2,0,1).cuda()
      image = fn.resize(image_og,[320,320]).unsqueeze(0).float().cuda()
      d1,*_= net(image)
      mask3 = fn.resize(d1[:,0,:,:] > 0.3,size)

      segmented3d = torch.masked_fill(image_og,mask3==0,value=0)
      #cv2_imshow(segmented3d)
      #print(get_pred(ocr_prediction(segmented3d)))
      #====
      transform = ToPILImage()
      image = transform(segmented3d[[2,1,0],:,:])
      ocr_text, image_np = ocr_prediction(image)
      brand_name = get_pred(ocr_text)
      #====
      #segmented3d = segmented3d.permute(1,2,0)[:,:,[2,1,0]].cpu().numpy()
      #cv2_imshow(segmented3d)
      # np.resize(image_np, (30, 1080, 1920//2, 3))
      # print(image_np.size)
      #image_np = image_np.resize()
      # text = brand_name + "\n" + " ".join(ocr_text)
      # brand_seg = cv2.putText(image_np, brand_name, (100,650),
      #                              cv2.FONT_HERSHEY_SIMPLEX,2,(57,255,20),2,cv2.LINE_AA)
      batch_segment[c] = cv2.putText(image_np, " ".join(ocr_text), (100,700),
                                    cv2.FONT_HERSHEY_SIMPLEX,1,(57,255,20),2,cv2.LINE_AA)

    np.save("/content/Batch_segment_only_ocr", batch_segment)
    #create_mask_video(batch_segment)
    #final_join_video()
      # return batch_segment

In [ ]:
def ocr_prediction(img):
    '''
    This function performs the OCR predictions using Glass Text Spotting by Amazon Science

    :param: img
    :param: second
    :return: nothing
    '''

    image = np.asarray(img.convert('RGB'))
    # cv2_imshow(image)
    rgb_image = image
    try:
      preds = glass_runner(image)
      text, text_scores, _ = get_instances_text(text_probs=preds.pred_text_prob, text_encoder=glass_runner.text_encoder, onlyRemoveFirstLastCharacter=False)
      return text, rgb_image
      # print("OCR Text:", text)
      # print("Fuzzy Text:", get_pred(text))
    except AttributeError:
      # print([])
      return [], rgb_image

In [ ]:
def get_closest_match(text):
    string = extractOne(text, adBoardItems, scorer=partial_ratio,score_cutoff=92)
    if string is not None:
        return string[0]
    return None


def get_brand(text):
    match = get_closest_match(text)
    if match is not None:
        return database[match]
    return None


def get_matches(text):
    return extract(text, adBoardItems)

def get_pred(lst):
  temp = []
  if len(lst):
    for item in lst:
      brand = get_brand(item.lower())
      if brand is not None and len(item)>2:
        temp.append(brand)

  return mode(temp) if len(temp) else " "

In [ ]:
def create_mask_video(segment_batch):
  batch_seg_fps = np.repeat(segment_batch,30,axis=0)
  print(batch_seg_fps.shape)

  video = cv2.VideoWriter('/content/mask_video_fps.mp4',cv2.VideoWriter_fourcc(*'mp4v'), 30, (1920,1080))

  for i in batch_seg_fps:
    video.write(i)
  video.release()

In [ ]:
def final_join_video():
  cap1 = cv2.VideoCapture(video_path)
  cap2 = cv2.VideoCapture("/content/mask_video_fps.mp4")

  BG = np.zeros((1080, 1920*2, 3), dtype = np.uint8)

  def Fram_connect(frame1, frame2, H, W, BG):
    BG[0:H,0:W] = frame1
    BG[0:H,W:W*2] = frame2
    return cv2.resize(BG,(W,H),interpolation = cv2.INTER_AREA)


  fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
  videowriter = cv2.VideoWriter("/content/output_video" + ".avi",fourcc,30,(1920,1080))

  while (True):
    _, frame1=cap1.read()
    _, frame2=cap2.read()
    videowriter.write(Fram_connect(frame1, frame2, 1080, 1920,BG))

In [ ]:
glass_model_path = './models/glass_icdar15.pth'
config_path = './configs/glass_finetune_icdar15.yaml'
glass_runner = GlassRunner(model_path=glass_model_path, config_path=config_path, post_process=False)

In [ ]:
import time
# import extract_frame

start_time = time.time()

videoStartAt = 0

video = cv2.VideoCapture(video_path)
video.set(1,videoStartAt)

fps = video.get(cv2.CAP_PROP_FPS)
totalNoFrames = video.get(cv2.CAP_PROP_FRAME_COUNT)
videoDurationInSeconds = totalNoFrames // fps

## for testing, performing inference for 14 seconds of video
videoLength = 600
extract_frame(video, fps, videoStartAt+videoLength)

end_time = time.time()

print("Time taken for inference of {} seconds of video  is {}s".format(videoLength, end_time - start_time))

Inference 23 second se chalu karenge, for video
do it for 2 mins, i.e., 120 secs

In [ ]:
import time
# import extract_frame

start_time = time.time()

videoStartAt = 0

video = cv2.VideoCapture(video_path)
video.set(1,videoStartAt)

fps = video.get(cv2.CAP_PROP_FPS)
totalNoFrames = video.get(cv2.CAP_PROP_FRAME_COUNT)
videoDurationInSeconds = totalNoFrames // fps

## for testing, performing inference for 14 seconds of video
videoLength = 120
extract_frame(video, fps, videoStartAt+videoLength)

end_time = time.time()

print("Time taken for inference of {} seconds of video  is {}s".format(videoLength, end_time - start_time))

Time taken for inference of 120 seconds of video  is 138.24648141860962s


In [ ]:
import time
# import extract_frame

start_time = time.time()

videoStartAt = 0

video = cv2.VideoCapture(video_path)
video.set(1,videoStartAt)

fps = video.get(cv2.CAP_PROP_FPS)
totalNoFrames = video.get(cv2.CAP_PROP_FRAME_COUNT)
videoDurationInSeconds = totalNoFrames // fps

## for testing, performing inference for 14 seconds of video
videoLength = 240
extract_frame(video, fps, videoStartAt+videoLength)

end_time = time.time()

print("Time taken for inference of {} seconds of video  is {}s".format(videoLength, end_time - start_time))

Time taken for inference of 240 seconds of video  is 294.39005398750305s


In [ ]:
import time
# import extract_frame

start_time = time.time()

videoStartAt = 0

video = cv2.VideoCapture(video_path)
video.set(1,videoStartAt)

fps = video.get(cv2.CAP_PROP_FPS)
totalNoFrames = video.get(cv2.CAP_PROP_FRAME_COUNT)
videoDurationInSeconds = totalNoFrames // fps

## for testing, performing inference for 14 seconds of video
videoLength = 480
extract_frame(video, fps, videoStartAt+videoLength)

end_time = time.time()

print("Time taken for inference of {} seconds of video  is {}s".format(videoLength, end_time - start_time))

Time taken for inference of 480 seconds of video  is 543.8926026821136s


In [ ]:
import time
# import extract_frame

start_time = time.time()

videoStartAt = 0

video = cv2.VideoCapture(video_path)
video.set(1,videoStartAt)

fps = video.get(cv2.CAP_PROP_FPS)
totalNoFrames = video.get(cv2.CAP_PROP_FRAME_COUNT)
videoDurationInSeconds = totalNoFrames // fps

## for testing, performing inference for 14 seconds of video
videoLength = 600
extract_frame(video, fps, videoStartAt+videoLength)

end_time = time.time()

print("Time taken for inference of {} seconds of video  is {}s".format(videoLength, end_time - start_time))

Time taken for inference of 600 seconds of video  is 678.9105579853058s


In [ ]:
#import time

#start_time = time.time()

videoStartAt = 24

video = cv2.VideoCapture(video_path)
#print(video.get(cv2.CAP_PROP_POS_FRAMES))
# video.set(videoStartAt*, videoStartAt)
#print(video.get(cv2.CAP_PROP_POS_FRAMES))


fps    = video.get(cv2.CAP_PROP_FPS)
width  = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

totalNoFrames = video.get(cv2.CAP_PROP_FRAME_COUNT)
videoDurationInSeconds = totalNoFrames // fps
videoLength = 120
video.set(1, videoStartAt*int(fps))

extract_frame(video, fps, videoStartAt, videoStartAt+videoLength, [height,width])
#end_time = time.time()

#print("Time taken for inference of {} minute of video  is {}s".format(videoDurationInSeconds/60, end_time - start_time))

144
